In [7]:
import pandas as pd
import glob
import numpy as np
import os

print(os.listdir())  # lists files in current directory

['smoglens', 'jp_openmeteo_processed_20230101_to_20231231.csv:Zone.Identifier', 'jp_era5_processed_20230101_to_20231231.csv', 'jp_nasapower_processed_20230101_to_20231231.csv', 'jp_jartic_processed_20230101_to_20231231.csv:Zone.Identifier', 'jp_jartic_processed_20230101_to_20231231.csv', '.ipynb_checkpoints', 'jp_terrain_processed_20230101_to_20231231.csv', 'jp_openaq_processed_20230101_to_20231231.csv', 'jp_openaq_processed_20230101_to_20231231.csv:Zone.Identifier', '2023smoglens_datapreprocessed_EDA.ipynb', 'jp_terrain_processed_20230101_to_20231231.csv:Zone.Identifier', 'DATA_DICTIONARY.md:Zone.Identifier', 'jp_era5_processed_20230101_to_20231231.csv:Zone.Identifier', 'jp_openmeteo_processed_20230101_to_20231231.csv', 'DATA_DICTIONARY.md', 'jp_nasapower_processed_20230101_to_20231231.csv:Zone.Identifier']


In [6]:
dfs['jp_era5_processed_20230101_to_20231231.csv'].head()


NameError: name 'dfs' is not defined

In [8]:
csv_files = glob.glob("*.csv")

In [3]:
print("Files found:")
for file in csv_files:
    print(file)

Files found:
jp_era5_processed_20230101_to_20231231.csv
jp_nasapower_processed_20230101_to_20231231.csv
jp_jartic_processed_20230101_to_20231231.csv
jp_terrain_processed_20230101_to_20231231.csv
jp_openaq_processed_20230101_to_20231231.csv
jp_openmeteo_processed_20230101_to_20231231.csv


In [10]:
files = [
    'jp_era5_processed_20230101_to_20231231.csv',
    'jp_nasapower_processed_20230101_to_20231231.csv',
    'jp_jartic_processed_20230101_to_20231231.csv',
    'jp_terrain_processed_20230101_to_20231231.csv',
    'jp_openaq_processed_20230101_to_20231231.csv',
    'jp_openmeteo_processed_20230101_to_20231231.csv'
]

dfs = {}
for file in files:
    if os.path.exists(file):
        dfs[file] = pd.read_csv(file)
    else:
        print(f"File not found: {file}")



In [5]:
df_sample = pd.read_csv(file, nrows=100000)  # adjust number of rows
duplicates_count = df_sample.duplicated().sum()
print(f"Sampled {file}: Number of duplicate rows = {duplicates_count}")


Sampled jp_openmeteo_processed_20230101_to_20231231.csv: Number of duplicate rows = 0


In [6]:
import gc

del df_sample  # or del chunk if using chunks
gc.collect()


0

**Missing Values**

In [7]:
for file, df in dfs.items():
    # Check missing (NaNs)
    missing_pct = df.isnull().mean()
    # Check empty strings (only for object columns)
    empty_str_pct = (df.select_dtypes(include='object') == '').mean()
    
    print(f"\nMissing values (NaN) for {file}:")
    print(missing_pct[missing_pct > 0])
    print(f"\nEmpty strings ('') for {file}:")
    print(empty_str_pct[empty_str_pct > 0])



Missing values (NaN) for jp_era5_processed_20230101_to_20231231.csv:
Series([], dtype: float64)

Empty strings ('') for jp_era5_processed_20230101_to_20231231.csv:
Series([], dtype: float64)

Missing values (NaN) for jp_nasapower_processed_20230101_to_20231231.csv:
nasa_cloud_cover_pct_mean         0.00615
nasa_cloud_cover_pct_min          0.00615
nasa_cloud_cover_pct_max          0.00615
nasa_cloud_cover_pct_count        0.00615
nasa_dew_point_c_mean             0.00615
nasa_dew_point_c_min              0.00615
nasa_dew_point_c_max              0.00615
nasa_dew_point_c_count            0.00615
nasa_humidity_pct_mean            0.00615
nasa_humidity_pct_min             0.00615
nasa_humidity_pct_max             0.00615
nasa_humidity_pct_count           0.00615
nasa_precipitation_mm_mean        0.00615
nasa_precipitation_mm_min         0.00615
nasa_precipitation_mm_max         0.00615
nasa_precipitation_mm_count       0.00615
nasa_pressure_hpa_mean            0.00615
nasa_pressure_hpa_m

**Scaling**

In [ ]:
from sklearn.impute import SimpleImputer

for file, df in dfs.items():
    # Calculate missing value percentages for this dataframe
    missing_pct = df.isnull().mean()

    # Select numerical columns
    num_cols = df.select_dtypes(include='number').columns

    # Reindex to keep only numerical columns and fill NaNs with 0 (meaning no missing)
    num_missing_pct = missing_pct.reindex(num_cols).fillna(0)

    # Check if there are missing values in numerical columns
    if num_missing_pct.sum() > 0:
        imputer = SimpleImputer(strategy='median')
        df[num_cols] = imputer.fit_transform(df[num_cols])
        print(f"{file}: Missing numerical values imputed with median.")
    else:
        print(f"{file}: No missing numerical values to impute.")

    # Save back updated dataframe
    dfs[file] = df


jp_era5_processed_20230101_to_20231231.csv: No missing numerical values to impute.
jp_nasapower_processed_20230101_to_20231231.csv: Missing numerical values imputed with median.
jp_jartic_processed_20230101_to_20231231.csv: Missing numerical values imputed with median.
jp_terrain_processed_20230101_to_20231231.csv: Missing numerical values imputed with median.


**Encode features**

In [ ]:
cat_cols = df.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded = ohe.fit_transform(df[cat_cols])

import numpy as np
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(cat_cols))

df = df.drop(columns=cat_cols).reset_index(drop=True)
df = pd.concat([df, encoded_df], axis=1)


**Cyclical engineering**